In [43]:
from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Helpers ---------- #
REL_PATH = Path("InputData/CoreData.xlsx")

def find_upwards(rel_path: Path, max_up: int = 8) -> Path:
    here = Path.cwd()
    for parent in [here, *here.parents][: max_up + 1]:
        candidate = (parent / rel_path)
        if candidate.exists():
            return candidate.resolve()
    raise FileNotFoundError(
        f"Couldn't locate '{rel_path.as_posix()}' from {here} by walking up {max_up} levels.\n"
        f"- Current working directory: {here}\n"
        f"- Checked: {[str((p / rel_path)) for p in [here, *here.parents][: max_up + 1]]}"
    )

# ---------- Step 1: Read Excel / basic checks ---------- #
INPUT_XLSX = find_upwards(REL_PATH)
xfile = pd.ExcelFile(INPUT_XLSX)
sheets = xfile.sheet_names
print("Resolved path:", INPUT_XLSX)
print("Sheets:", sheets)

assert isinstance(sheets, list) and sheets, "Sheet names missing."
assert all(isinstance(s, str) and s.strip() for s in sheets), "Invalid sheet name(s)."
assert len(sheets) == len(set(sheets)), "Duplicate sheet names detected."

SHEET = "deal_cash_flow"
assert SHEET in sheets, f"'{SHEET}' not found. Available sheets: {sheets}"

# Load only the header first to inspect columns safely
probe = pd.read_excel(INPUT_XLSX, sheet_name=SHEET, nrows=0)
cols = list(probe.columns)
print("Columns:", cols)
assert "id" in cols, f"'id' column not found in '{SHEET}'."

# ---------- Step 2: Initialize GrossToNet/Data/deal_cash_flow.csv with id ---------- #
TARGET_DIR = (find_upwards(Path("GrossToNet")) / "Data")
TARGET_DIR.mkdir(parents=True, exist_ok=True)
TARGET_CSV = TARGET_DIR / "deal_cash_flow.csv"

raw = pd.read_excel(INPUT_XLSX, sheet_name=SHEET, usecols=["id"], dtype={"id": str})
df = raw[["id"]]
df.to_csv(TARGET_CSV, index=False)
print(f"Wrote {len(df):,} rows to {TARGET_CSV}")

# ---------- Step 3: Integrity checks on the written CSV ---------- #
assert TARGET_CSV.exists(), f"Missing output: {TARGET_CSV}"
check_df = pd.read_csv(TARGET_CSV, dtype={"id": str})

assert list(check_df.columns) == ["id"], f"Unexpected columns: {list(check_df.columns)}"
assert len(check_df) == len(raw), f"Row count changed: raw={len(raw)} vs written={len(check_df)}"
assert check_df["id"].tolist() == raw["id"].tolist(), "Row order changed."
assert check_df["id"].notna().all(), "Null id found."
assert not check_df["id"].duplicated().any(), "Duplicate id values found."

print("INIT (deal_cash_flow id seed) check passed. Shape:", check_df.shape)

# ---------- Step 3: Add columns from deal_cash_flow ---------- #
TARGET_CSV = (find_upwards(Path("GrossToNet")) / "Data" / "deal_cash_flow.csv")
working = pd.read_csv(TARGET_CSV, dtype={"id": str})

requested = ["cash_flow_type", "currency", "cash_flow", "cash_flow_date", "deal_id"]

src = pd.read_excel(
    INPUT_XLSX,
    sheet_name="deal_cash_flow",
    usecols=["id", *requested],
    dtype={"id": str},
)

# Ensure one row per id to avoid row-multiplying merges.
assert src["id"].is_unique, "deal_cash_flow: duplicate id values would explode rows on merge."

# Normalize cash_flow_date if present (handle Excel serials and strings).
if "cash_flow_date" in src.columns:
    s = src["cash_flow_date"]
    dt = pd.to_datetime(s, errors="coerce")
    ser = pd.to_numeric(s, errors="coerce")
    # Heuristic: many Excel serials are > 20000
    is_serialish = (dt.isna() & ser.gt(20000)).mean() > 0.5
    if is_serialish:
        dt = pd.to_datetime(ser, unit="D", origin="1899-12-30", errors="coerce")
    src["cash_flow_date"] = dt.dt.strftime("%Y-%m-%d")

to_add = [c for c in requested if c not in working.columns]
src = src[["id", *to_add]]

working["_ord"] = np.arange(len(working))
out = working.merge(src, on="id", how="left")
out = out.sort_values("_ord").drop(columns="_ord")
out.to_csv(TARGET_CSV, index=False)
print(f"Added columns from 'deal_cash_flow': {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

# Post-merge checks
after = pd.read_csv(TARGET_CSV, dtype={"id": str})
assert len(after) == len(working), "Row count changed."
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Order changed."
missing = [c for c in to_add if c not in after.columns]
assert not missing, f"Missing columns after merge: {missing}"
_ = pd.to_datetime(after["cash_flow_date"], errors="coerce")  # smoke-check
print("ADD_COLUMNS (deal_cash_flow) check passed. Shape:", after.shape)


Resolved path: /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/InputData/CoreData.xlsx
Sheets: ['Metadata', 'dashboard', 'general_partner', 'fund', 'fund_cash_flow', 'capital_account', 'deal', 'deal_time_series', 'deal_cash_flow', 'deal_partner', 'deal_acquirer', 'deal_vendor', 'organization', 'person']
Columns: ['id', 'notes', 'cash_flow_type', '_created_at_utc', 'adjustment', 'data_room_id', 'created_by_user_id', 'cash_flow_subtype', 'currency', '_revision_id', 'cash_flow', 'cash_flow_date', 'deal_revision_id', 'deal_id', 'data_room_name']
Wrote 14,289 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/GrossToNet/Data/deal_cash_flow.csv
INIT (deal_cash_flow id seed) check passed. Shape: (14289, 1)
Added columns from 'deal_cash_flow': ['cash_flow_type', 'currency', 'cash_flow', 'cash_flow_date', 'deal

In [44]:
# ---------- Build fund_cash_flow.csv (init + add columns) ---------- #
TARGET_DIR = (find_upwards(Path("GrossToNet")) / "Data")
TARGET_DIR.mkdir(parents=True, exist_ok=True)
TARGET_CSV = TARGET_DIR / "fund_cash_flow.csv"

SHEET_FUND_CF = "fund_cash_flow"
assert SHEET_FUND_CF in sheets, f"'{SHEET_FUND_CF}' not found. Available sheets: {sheets}"

# Init with id
raw_id = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_FUND_CF, usecols=["id"], dtype={"id": str})
df_init = raw_id[["id"]]
df_init.to_csv(TARGET_CSV, index=False)
print(f"[fund_cf] Wrote {len(df_init):,} id rows to {TARGET_CSV}")

# Verify init
check = pd.read_csv(TARGET_CSV, dtype={"id": str})
assert list(check.columns) == ["id"], f"Unexpected columns: {list(check.columns)}"
assert len(check) == len(df_init), "Row count changed at init."
assert check["id"].tolist() == df_init["id"].tolist(), "Order changed at init."
assert check["id"].notna().all(), "Null id at init."
assert not check["id"].duplicated().any(), "Duplicate id at init."
print("[fund_cf] INIT check passed.", check.shape)

# Add columns from fund_cash_flow
working = pd.read_csv(TARGET_CSV, dtype={"id": str})
requested = ["cash_flow_type", "fund_id", "cash_flow", "cash_flow_date"]

src = pd.read_excel(
    INPUT_XLSX,
    sheet_name=SHEET_FUND_CF,
    usecols=["id", *requested],
    dtype={"id": str},
)

# Ensure one row per id
assert src["id"].is_unique, "fund_cash_flow: duplicate id values would explode rows on merge."

# Normalize cash_flow_date (handle strings and Excel serials)
if "cash_flow_date" in src.columns:
    s = src["cash_flow_date"]
    dt = pd.to_datetime(s, errors="coerce")
    ser = pd.to_numeric(s, errors="coerce")
    is_serialish = (dt.isna() & ser.gt(20000)).mean() > 0.5
    if is_serialish:
        dt = pd.to_datetime(ser, unit="D", origin="1899-12-30", errors="coerce")
    src["cash_flow_date"] = dt.dt.strftime("%Y-%m-%d")

to_add = [c for c in requested if c not in working.columns]
src = src[["id", *to_add]]

working["_ord"] = np.arange(len(working))
out = working.merge(src, on="id", how="left")
out = out.sort_values("_ord").drop(columns="_ord")
out.to_csv(TARGET_CSV, index=False)
print(f"[fund_cf] Added columns: {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

# Post-merge checks
after = pd.read_csv(TARGET_CSV, dtype={"id": str})
assert len(after) == len(working), "Row count changed after merge."
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Order changed after merge."
missing = [c for c in to_add if c not in after.columns]
assert not missing, f"Missing columns after merge: {missing}"
_ = pd.to_datetime(after["cash_flow_date"], errors="coerce")
print("[fund_cf] ADD_COLUMNS check passed. Shape:", after.shape)

# ---------- Enrich fund_cash_flow.csv with fund_currency from fund sheet ---------- #
TARGET_CSV = (find_upwards(Path("GrossToNet")) / "Data" / "fund_cash_flow.csv")

# Load current fund_cash_flow
working = pd.read_csv(TARGET_CSV, dtype={"id": str, "fund_id": str})

# Pull mapping: fund.id -> fund.reported_currency
fund_map = (
    pd.read_excel(
        INPUT_XLSX,
        sheet_name="fund",
        usecols=["id", "reported_currency"],
        dtype={"id": str},
    )
    .rename(columns={"id": "fund_id", "reported_currency": "fund_currency"})
)

# Robustness: ensure one row per fund_id in mapping
assert fund_map["fund_id"].notna().all(), "Null fund_id in fund sheet."
assert fund_map["fund_id"].is_unique, "Duplicate fund_id in fund sheet; cannot map currency uniquely."

# Normalize currency text
fund_map["fund_currency"] = fund_map["fund_currency"].astype(str).str.strip().str.upper()
fund_map.loc[fund_map["fund_currency"].isin(["", "NAN", "NONE"]), "fund_currency"] = pd.NA

# Merge while preserving row order
to_add = ["fund_currency"]
working["_ord"] = np.arange(len(working))
out = (
    working.merge(fund_map[["fund_id", "fund_currency"]], on="fund_id", how="left")
           .sort_values("_ord")
           .drop(columns="_ord")
)

# Write back
out.to_csv(TARGET_CSV, index=False)
print(f"[fund_cf] Added column: {to_add}. Wrote {len(out):,} rows to {TARGET_CSV}.")

# -------- QA -------- #
after = pd.read_csv(TARGET_CSV, dtype={"id": str, "fund_id": str})

# 1) Column exists
assert "fund_currency" in after.columns, "fund_currency missing after merge."

# 2) Coverage: all non-null fund_id should have a currency
missing_cur = after[after["fund_id"].notna() & after["fund_id"].astype(str).str.strip().ne("") & after["fund_currency"].isna()]
assert missing_cur.empty, f"Missing fund_currency for some fund_id values:\n{missing_cur['fund_id'].value_counts().head()}"

# 3) Basic validity: currency looks like ISO code (3 letters)
non_iso = after["fund_currency"].dropna().astype(str).str.fullmatch(r"[A-Z]{3}") == False
assert (~non_iso).all(), "Some fund_currency values are not 3-letter codes."

# 4) Order preserved and row count unchanged
assert len(after) == len(working), "Row count changed."
assert after["id"].tolist() == working.sort_values("_ord")["id"].tolist(), "Row order changed."

# Summary
print("[fund_cf] Currency coverage:",
      int(after["fund_currency"].notna().sum()), "/", len(after))
print("[fund_cf] Unique currencies:", sorted(after["fund_currency"].dropna().unique().tolist()))


[fund_cf] Wrote 7,563 id rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/GrossToNet/Data/fund_cash_flow.csv
[fund_cf] INIT check passed. (7563, 1)
[fund_cf] Added columns: ['cash_flow_type', 'fund_id', 'cash_flow', 'cash_flow_date']. Wrote 7,563 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/GrossToNet/Data/fund_cash_flow.csv.
[fund_cf] ADD_COLUMNS check passed. Shape: (7563, 5)
[fund_cf] Added column: ['fund_currency']. Wrote 7,563 rows to /Users/michael/Library/Mobile Documents/com~apple~CloudDocs/Studium TUM/Master Management and Technology/06 Master Thesis/00 Thesis/05Code/GrossToNet/Data/fund_cash_flow.csv.
[fund_cf] Currency coverage: 7563 / 7563
[fund_cf] Unique currencies: ['DKK', 'EUR', 'GBP', 'SEK', 'USD']


In [45]:
# ---------- Clean cash-flow CSVs by group-level rules + QA ---------- #
from pathlib import Path
import pandas as pd
import numpy as np

DATA_DIR = (find_upwards(Path("GrossToNet")) / "Data")
DEALS_CSV = DATA_DIR / "deal_cash_flow.csv"
FUNDS_CSV = DATA_DIR / "fund_cash_flow.csv"

# Dates
MIN_DATE = pd.Timestamp("1976-01-01")
TODAY    = pd.Timestamp.today().normalize()  # uses system local time

# Allowed types
ALLOWED_DEAL_TYPES = {"investment", "proceed", "fair value"}
ALLOWED_FUND_TYPES = {"contribution", "distribution", "nav"}

# --- Load ---
deals = pd.read_csv(DEALS_CSV, dtype={"id": str, "deal_id": str, "currency": str})
funds = pd.read_csv(FUNDS_CSV, dtype={"id": str, "fund_id": str})

# Parse dates (tolerate bad values; we’ll filter them)
deals["cash_flow_date"] = pd.to_datetime(deals["cash_flow_date"], format="%Y-%m-%d", errors="coerce")
funds["cash_flow_date"] = pd.to_datetime(funds["cash_flow_date"], format="%Y-%m-%d", errors="coerce")

# --- Row-level invalidity flags (then escalate to group-level) ---
# Deals: drop entire deal_id if ANY row violates
deal_type_std = deals["cash_flow_type"].astype(str).str.strip().str.lower()
deal_invalid_type = ~deal_type_std.isin(ALLOWED_DEAL_TYPES)
deal_missing_required = (
    deals["currency"].isna()
    | deals["cash_flow"].isna()
    | deals["cash_flow_date"].isna()
)
deal_invalid_date = (~deals["cash_flow_date"].between(MIN_DATE, TODAY)) | deals["cash_flow_date"].isna()
deal_missing_deal_id = deals["deal_id"].isna() | (deals["deal_id"].astype(str).str.strip() == "")

# --- Early-history cutoff for deals (drop whole deal_id group if any row < 1992-07-01) ---
CUTOFF = pd.Timestamp("1992-07-01")

assert pd.api.types.is_datetime64_any_dtype(deals["cash_flow_date"]), "deals.cash_flow_date must be datetime"
pre_mask = deals["cash_flow_date"] < CUTOFF
early_deal_ids = deals.loc[pre_mask, "deal_id"].dropna().unique().tolist()

if early_deal_ids:
    before_rows   = len(deals)
    before_groups = deals["deal_id"].nunique(dropna=False)
    deals = deals[~deals["deal_id"].isin(early_deal_ids)].copy()
    after_rows    = len(deals)
    after_groups  = deals["deal_id"].nunique(dropna=False)
    print(f"[CUTOFF] Removed {len(early_deal_ids)} deal_id groups with any date < {CUTOFF.date()} "
          f"({before_groups} -> {after_groups}); rows {before_rows} -> {after_rows}.")
else:
    print(f"[CUTOFF] No deal_id groups with dates < {CUTOFF.date()} found.")

deals["_row_invalid"] = (
    deal_invalid_type
    | deal_missing_required
    | deal_invalid_date
    | deal_missing_deal_id
)

# Funds: drop entire fund_id if ANY row violates
fund_type_std = funds["cash_flow_type"].astype(str).str.strip().str.lower()
fund_invalid_type = ~fund_type_std.isin(ALLOWED_FUND_TYPES)
fund_invalid_date = (~funds["cash_flow_date"].between(MIN_DATE, TODAY)) | funds["cash_flow_date"].isna()
fund_missing_fund_id = funds["fund_id"].isna() | (funds["fund_id"].astype(str).str.strip() == "")

# --- Early-history cutoff for funds (drop whole fund_id group if any row < 1992-07-01) ---
CUTOFF = pd.Timestamp("1992-07-01")  # reuse if already defined above

assert pd.api.types.is_datetime64_any_dtype(funds["cash_flow_date"]), "funds.cash_flow_date must be datetime"
pre_mask_f = funds["cash_flow_date"] < CUTOFF
early_fund_ids = funds.loc[pre_mask_f, "fund_id"].dropna().unique().tolist()

if early_fund_ids:
    f_before_rows   = len(funds)
    f_before_groups = funds["fund_id"].nunique(dropna=False)
    funds = funds[~funds["fund_id"].isin(early_fund_ids)].copy()
    f_after_rows    = len(funds)
    f_after_groups  = funds["fund_id"].nunique(dropna=False)
    print(f"[CUTOFF] Funds: removed {len(early_fund_ids)} fund_id groups with any date < {CUTOFF.date()} "
          f"({f_before_groups} -> {f_after_groups}); rows {f_before_rows} -> {f_after_rows}.")
else:
    print(f"[CUTOFF] Funds: no fund_id groups with dates < {CUTOFF.date()} found.")

funds["_row_invalid"] = fund_invalid_type | fund_invalid_date | fund_missing_fund_id

# --- Escalate to group-level removal ---
# Deals
invalid_deal_ids = (
    deals.groupby("deal_id", dropna=False)["_row_invalid"]
    .any()
    .pipe(lambda s: s[s].index.tolist())
)
deals_before_rows = len(deals)
deals_before_groups = deals["deal_id"].nunique(dropna=False)

deals = deals[~deals["deal_id"].isin(invalid_deal_ids)].copy()

# Funds
invalid_fund_ids = (
    funds.groupby("fund_id", dropna=False)["_row_invalid"]
    .any()
    .pipe(lambda s: s[s].index.tolist())
)
funds_before_rows = len(funds)
funds_before_groups = funds["fund_id"].nunique(dropna=False)

funds = funds[~funds["fund_id"].isin(invalid_fund_ids)].copy()

# Drop helper flags
for df in (deals, funds):
    if "_row_invalid" in df.columns:
        df.drop(columns=["_row_invalid"], inplace=True)

# --- Write back cleaned CSVs ---
deals.to_csv(DEALS_CSV, index=False)
funds.to_csv(FUNDS_CSV, index=False)

# --- QA checks and summaries ---
# Assert remaining types are allowed
assert set(deals["cash_flow_type"].astype(str).str.strip().str.lower().unique()).issubset(ALLOWED_DEAL_TYPES), \
    "Unexpected deal cash_flow_type after cleaning."
assert set(funds["cash_flow_type"].astype(str).str.strip().str.lower().unique()).issubset(ALLOWED_FUND_TYPES), \
    "Unexpected fund cash_flow_type after cleaning."

# Assert required fields present and in range
assert deals["currency"].notna().all(), "Missing currency in deals after cleaning."
assert deals["cash_flow"].notna().all(), "Missing cash_flow in deals after cleaning."
assert deals["cash_flow_date"].notna().all(), "Missing cash_flow_date in deals after cleaning."
assert deals["cash_flow_date"].between(MIN_DATE, TODAY).all(), "Deals dates out of range after cleaning."
assert funds["cash_flow_date"].notna().all(), "Missing cash_flow_date in funds after cleaning."
assert funds["cash_flow_date"].between(MIN_DATE, TODAY).all(), "Funds dates out of range after cleaning."

# Print removals
print("=== Removals ===")
print(f"Deals: removed {len(invalid_deal_ids)} deal_id groups "
      f"({deals_before_groups} -> {deals['deal_id'].nunique(dropna=False)}); "
      f"rows {deals_before_rows} -> {len(deals)}.")
if invalid_deal_ids:
    print("Sample removed deal_ids:", invalid_deal_ids[:10])

print(f"Funds: removed {len(invalid_fund_ids)} fund_id groups "
      f"({funds_before_groups} -> {funds['fund_id'].nunique(dropna=False)}); "
      f"rows {funds_before_rows} -> {len(funds)}.")
if invalid_fund_ids:
    print("Sample removed fund_ids:", invalid_fund_ids[:10])

# Counts by type
deal_type_counts = deals["cash_flow_type"].str.strip().str.lower().value_counts()
fund_type_counts = funds["cash_flow_type"].str.strip().str.lower().value_counts()

print("\n=== Type counts (deals) ===")
print(deal_type_counts.to_string())
print("=== Type counts (funds) ===")
print(fund_type_counts.to_string())

# Date ranges
def safe_range(s: pd.Series):
    if s.empty:
        return (None, None)
    return (s.min(), s.max())

dmin, dmax = safe_range(deals["cash_flow_date"])
fmin, fmax = safe_range(funds["cash_flow_date"])
print("\n=== Date ranges ===")
print(f"Deals: {dmin} .. {dmax}")
print(f"Funds: {fmin} .. {fmax}")

# Additional sanity
print("\n=== Sanity ===")
print({"deal_rows": len(deals), "deal_groups": deals['deal_id'].nunique(),
       "fund_rows": len(funds), "fund_groups": funds['fund_id'].nunique()})


[CUTOFF] Removed 6 deal_id groups with any date < 1992-07-01 (1261 -> 1255); rows 14289 -> 14209.
[CUTOFF] Funds: removed 1 fund_id groups with any date < 1992-07-01 (145 -> 144); rows 7563 -> 7533.
=== Removals ===
Deals: removed 277 deal_id groups (1255 -> 978); rows 14209 -> 11992.
Sample removed deal_ids: ['0178d460-b750-4b65-944a-35dd5647a51f', '0222e065-cd3d-4d0c-8bc9-266adf7a2054', '025a601e-f0a4-42a0-b579-8572833bf692', '036f81c2-d206-4134-9e90-ba2cbf5bdb45', '03e00158-66c9-4113-8ba8-ab827818aee5', '06e56493-ab4c-4746-b6fa-f0786a52af98', '0a086ab3-b0ce-4e60-9877-096eaf468422', '0a42cf0a-f224-4c25-ac9c-510e44466db4', '0ab3dea2-ceb2-4f3b-afdc-6f69b1eea789', '0b205007-0b03-4ab9-9549-3734b4b559fa']
Funds: removed 3 fund_id groups (144 -> 141); rows 7533 -> 7300.
Sample removed fund_ids: ['48b60287-d48d-4b8a-ae4c-7ebcedc99707', 'e2a37bbd-7670-4202-9cc8-81c0822ce08c', 'f6854930-2707-4825-958b-39a3eba5cf4f']

=== Type counts (deals) ===
cash_flow_type
proceed       5977
investment    